# Problem #1

In [1]:
import pandas as pd
import os
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
#import sys                                                    #Causes problems with regression in skikitlearn 
#reload(sys)                                                   #later on. Find different way to make Count Vectorizer
#sys.setdefaultencoding('utf8')                                #work. In future be carefull making global changes
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.linear_model import ARDRegression
from sklearn.ensemble import BaggingRegressor

C:\Users\user\Anaconda2\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
os.chdir('C:\Users\user\downloads')
salaryfn = 'salary_uk.csv'
salary_uk = pd.read_csv(salaryfn)

# Preprocessing

In [3]:
#drop redundant raw location and raw salary features
salary_uk = salary_uk.drop([u'LocationRaw', u'SalaryRaw', u'SourceName'],axis =1)

In [4]:
salary_uk.head()

,Id,Title,FullDescription,LocationNormalized,ContractType,ContractTime,Company,Category,SalaryNormalized
0,12612628,Engineering Systems Analyst,Engineering Systems Analyst Dorking Surrey Sal...,Dorking,NaN,permanent,Gregory Martin International,Engineering Jobs,25000
1,12612830,Stress Engineer Glasgow,Stress Engineer Glasgow Salary **** to **** We...,Glasgow,NaN,permanent,Gregory Martin International,Engineering Jobs,30000
2,12612844,Modelling and simulation analyst,Mathematical Modeller / Simulation Analyst / O...,Hampshire,NaN,permanent,Gregory Martin International,Engineering Jobs,30000
3,12613049,Engineering Systems Analyst / Mathematical Mod...,Engineering Systems Analyst / Mathematical Mod...,Surrey,NaN,permanent,Gregory Martin International,Engineering Jobs,27500
4,12613647,"Pioneer, Miser Engineering Systems Analyst","Pioneer, Miser Engineering Systems Analyst Do...",Surrey,NaN,permanent,Gregory Martin International,Engineering Jobs,25000


In [5]:
salary_uk.columns

Index([u'Id', u'Title', u'FullDescription', u'LocationNormalized',
       u'ContractType', u'ContractTime', u'Company', u'Category',
       u'SalaryNormalized'],
      dtype='object')

In [6]:
#Uses CountVectorizer, I use TfidVectorizer which is superior to CountVectorizer because it 
#uses term frequency-inverse document frequency (TF–IDF) which weights the word counts by a measure 
#of how often they appear in the documents

#vec = CountVectorizer()
#X = vec.fit_transform(salary_uk['Title'].values.astype('string'))
#Title = pd.DataFrame(X.toarray(), columns=vec.get_feature_names())

In [7]:
#Vectorizes Title feature using term frequency-inverse document frequency 
vec = TfidfVectorizer()
X = vec.fit_transform(salary_uk['Title'].values.astype('string'))
Title = pd.DataFrame(X.toarray(), columns=vec.get_feature_names())

In [ ]:
#Vectorizes FullDescription feature using term frequency-inverse document frequency 
vec = TfidfVectorizer()
X = vec.fit_transform(salary_uk['FullDescription'].values.astype('string'))
FullDescription = pd.DataFrame(X.toarray(), columns=vec.get_feature_names())

In [ ]:
salary_uk = pd.merge(salary_uk, Title, left_index=True, right_index=True)

In [ ]:
#This adds the FullDescription vector. It also makes the dataframe a little bit too big to run on my laptop so for now its
#commented out. Without this the datframe will have 3902 columns instead of 31087. The dataframe with 31087 columns causes
#memory error.

#salary_uk =pd.merge(salary_uk, FullDescription, left_index=True, right_index=True)

In [ ]:
salary_uk = salary_uk.drop(['Title','FullDescription'], axis = 1) #drops the now redundant columns

In [ ]:
salary_uk = pd.get_dummies(salary_uk, columns =[u'LocationNormalized',
       u'ContractType', u'ContractTime', u'Company', u'Category'])

In [ ]:
salary_uk.head()

Now we have our dataset ready for analysis.

# Exploratory Data Analysis 

# Partitioning the Data Set

In [ ]:
Y_salary_uk =salary_uk['SalaryNormalized']
X_salary_uk = salary_uk.drop('SalaryNormalized', axis=1)
Xtrain, Xtest, ytrain, ytest = train_test_split(X_salary_uk, Y_salary_uk,
                                                random_state=9)
Xcolnames = list(X_salary_uk)

In [ ]:
Y_salary_uk.shape

In [ ]:
X_salary_uk.shape

# Analysis

# Linear Regression

In [17]:
lr = LinearRegression().fit(Xtrain, ytrain)
print("Training set score: {:.2f}".format(lr.score(Xtrain, ytrain)))
print("Test set score: {:.2f}".format(lr.score(Xtest, ytest)))

Training set score: 0.88
Test set score: -11510814.38


# Ridge Regression

In [18]:
ridge = Ridge().fit(Xtrain, ytrain)
print("Training set score: {:.2f}".format(ridge.score(Xtrain, ytrain)))
print("Test set score: {:.2f}".format(ridge.score(Xtest, ytest)))

C:\Users\user\Anaconda2\lib\site-packages\sklearn\linear_model\ridge.py:112: LinAlgWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number5.576186e-19
  overwrite_a=True).T


Training set score: 0.79
Test set score: 0.57


In [19]:
ridge10 = Ridge(alpha=10).fit(Xtrain, ytrain)
print("Training set score: {:.2f}".format(ridge10.score(Xtrain, ytrain)))
print("Test set score: {:.2f}".format(ridge10.score(Xtest, ytest)))

C:\Users\user\Anaconda2\lib\site-packages\sklearn\linear_model\ridge.py:112: LinAlgWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number3.912640e-18
  overwrite_a=True).T


Training set score: 0.60
Test set score: 0.52


# K Nearest Neighbors Regressor

In [20]:
# instantiate the model and set the number of neighbors to consider to 3:
reg = KNeighborsRegressor(n_neighbors=3)
# fit the model using the training data and training targets:
reg.fit(Xtrain, ytrain)
print("Test set R^2: {:.2f}".format(reg.score(Xtest, ytest)))

Test set R^2: 0.09


In [21]:
# instantiate the model and set the number of neighbors to consider to 50:
reg = KNeighborsRegressor(n_neighbors=50)
# fit the model using the training data and training targets:
reg.fit(Xtrain, ytrain)
print("Test set R^2: {:.2f}".format(reg.score(Xtest, ytest)))

Test set R^2: 0.13


In [22]:
# instantiate the model and set the number of neighbors to consider to 150:
reg = KNeighborsRegressor(n_neighbors=150)
# fit the model using the training data and training targets:
reg.fit(Xtrain, ytrain)
print("Test set R^2: {:.2f}".format(reg.score(Xtest, ytest)))

Test set R^2: 0.08


# Lasso

In [23]:
lasso = Lasso().fit(Xtrain, ytrain)
print("Training set score: {:.2f}".format(lasso.score(Xtrain, ytrain)))
print("Test set score: {:.2f}".format(lasso.score(Xtest, ytest)))
print("Number of features used: {}".format(np.sum(lasso.coef_ != 0)))

Training set score: 0.80
Test set score: 0.55
Number of features used: 1893


In [29]:
lasso01 = Lasso(alpha=0.9, max_iter=1000).fit(Xtrain, ytrain)
print("Training set score: {:.2f}".format(lasso001.score(Xtrain, ytrain)))
print("Test set score: {:.2f}".format(lasso001.score(Xtest, ytest)))
print("Number of features used: {}".format(np.sum(lasso001.coef_ != 0)))

Training set score: 0.88
Test set score: 0.29
Number of features used: 4640


In [28]:
lasso01 = Lasso(alpha=0.1, max_iter=1000).fit(Xtrain, ytrain)
print("Training set score: {:.2f}".format(lasso001.score(Xtrain, ytrain)))
print("Test set score: {:.2f}".format(lasso001.score(Xtest, ytest)))
print("Number of features used: {}".format(np.sum(lasso001.coef_ != 0)))

Training set score: 0.88
Test set score: 0.29
Number of features used: 4640


In [27]:
# we increase the default setting of "max_iter",
# otherwise the model would warn us that we should increase max_iter.
lasso001 = Lasso(alpha=0.01, max_iter=1000).fit(Xtrain, ytrain)
print("Training set score: {:.2f}".format(lasso001.score(Xtrain, ytrain)))
print("Test set score: {:.2f}".format(lasso001.score(Xtest, ytest)))
print("Number of features used: {}".format(np.sum(lasso001.coef_ != 0)))

Training set score: 0.88
Test set score: 0.29
Number of features used: 4640


In [26]:
lasso00001 = Lasso(alpha=0.0001, max_iter=1000).fit(Xtrain, ytrain)
print("Training set score: {:.2f}".format(lasso00001.score(Xtrain, ytrain)))
print("Test set score: {:.2f}".format(lasso00001.score(Xtest, ytest)))
print("Number of features used: {}".format(np.sum(lasso00001.coef_ != 0)))

Training set score: 0.88
Test set score: 0.16
Number of features used: 5207


# Decision Tree Regressor

In [33]:
tree = DecisionTreeRegressor().fit(Xtrain, ytrain)
print("Training set score: {:.2f}".format(tree.score(Xtrain, ytrain)))
print("Test set score: {:.2f}".format(tree.score(Xtest, ytest)))

Training set score: 1.00
Test set score: 0.32


# Gradient Boosting Regressor

In [37]:
gbtr = GradientBoostingRegressor().fit(Xtrain, ytrain)
print("Training set score: {:.2f}".format(gbtr.score(Xtrain, ytrain)))
print("Test set score: {:.2f}".format(gbtr.score(Xtest, ytest)))

Training set score: 0.54
Test set score: 0.45


# Ada Boost Regressor

In [39]:
abtr = AdaBoostRegressor().fit(Xtrain, ytrain)
print("Training set score: {:.2f}".format(abtr.score(Xtrain, ytrain)))
print("Test set score: {:.2f}".format(abtr.score(Xtest, ytest)))

Training set score: -1.49
Test set score: -1.40


# Extra Trees Regressor

In [44]:
etr = ExtraTreesRegressor().fit(Xtrain, ytrain)
print("Training set score: {:.2f}".format(etr.score(Xtrain, ytrain)))
print("Test set score: {:.2f}".format(etr.score(Xtest, ytest)))

Training set score: 1.00
Test set score: 0.50


# Random Forest Regressor

In [45]:
rfr = RandomForestRegressor().fit(Xtrain, ytrain)
print("Training set score: {:.2f}".format(rfr.score(Xtrain, ytrain)))
print("Test set score: {:.2f}".format(rfr.score(Xtest, ytest)))

Training set score: 0.92
Test set score: 0.52


# Gaussian Process Regressor

In [ ]:
#takes a lil bit

#gpr = GaussianProcessRegressor().fit(Xtrain, ytrain)
#print("Training set score: {:.2f}".format(gpr.score(Xtrain, ytrain)))
#print("Test set score: {:.2f}".format(gpr.score(Xtest, ytest)))

# ARD Regression

In [ ]:
ardr = ARDRegression().fit(Xtrain, ytrain)
print("Training set score: {:.2f}".format(ardr.score(Xtrain, ytrain)))
print("Test set score: {:.2f}".format(ardr.score(Xtest, ytest)))